In [3]:
import os, sys
import json
import re
import time
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

import tfidf_searching
from clean_text import cleaned_text

In [79]:
import importlib
importlib.reload(tfidf_searching)

<module 'tfidf_searching' from '/home/chentianyu/libgen/tf-idf/tfidf_searching.py'>

In [4]:
with open('/home/chentianyu/dependency/dataset/train_reduced.json', 'r') as f:
    trains = json.load(f)
with open('/home/chentianyu/dependency/dataset/test_reduced.json', 'r') as f:
    tests = json.load(f)

In [5]:
oskg_folder = '/home/chentianyu/dependency/oskg/'
node_path = os.path.join(oskg_folder, 'oskg_node2os_20231120163318.csv')
edge_path = os.path.join(oskg_folder, 'oskg_edges_20231120163318.csv')
pros_path = os.path.join(oskg_folder, 'oskg_pros_20231120163318.csv')

nodes = pd.read_csv(node_path, delimiter='\t', header=None)
edges = pd.read_csv(edge_path, delimiter='\t', header=None)
pros = pd.read_csv(pros_path, delimiter='\t')

/tmp/ipykernel_3683027/2234550152.py:8: DtypeWarning: Columns (1,3,4,5,6,8,9,10,11,14,15,16,17,18,21,25,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  pros = pd.read_csv(pros_path, delimiter='\t')


In [6]:
pros = pros[pros.apply(lambda x: type(x['name']) == str, axis=1)]
pros_names = set([name.lower() for name in pros.name.to_list() if type(name) == str])

In [7]:
def get_c_artifact(lib):
    complete_name = ':'.join(lib.split(':')[1:])
    artifact = complete_name.split('/')[-1]
    return artifact.lower()

In [14]:
vulns = trains + tests
vuln_labels = [get_c_artifact(vuln['label']) for vuln in vulns]

In [8]:
name_weight = 4
pros_corpus = pros.drop_duplicates('name')[['name', 'summary']]
pros_corpus.columns = ['object', 'token']
pros_corpus.object = pros_corpus.object.apply(lambda x: x.lower())
pros_corpus.token = pros_corpus.token.apply(lambda x: x if type(x) == str else ' ')
pros_mapping = pros_corpus.set_index('object').to_dict()
pros_corpus.token = pros_corpus.apply(\
                    lambda x: f"{x['object'] * name_weight} {x['token']}", axis=1)

pros_corpus.token = pros_corpus.token.apply(lambda x: cleaned_text(x))
pros_corpus.token = pros_corpus.token.apply(lambda x: ' '.join(x))

In [ ]:
pros_corpus

In [10]:
search_engine = tfidf_searching.TfidfSearching(pros_corpus, 512, 2)

In [11]:
def recall(vuln, search_result, k=128):
    artifact = get_c_artifact(vuln['raw_label'])
    return artifact in search_result[:k]

In [12]:
def fun(vuln):
    search_engine = tfidf_searching.TfidfSearching(pros_corpus, 1024, 2)
    return search_engine.search_topk_objects(cleaned_text(vuln['desc']), [])

In [15]:
with Pool(processes=64) as pool:
    tf_idf_res = list(tqdm(pool.imap(fun, vulns)))

2292it [01:47, 21.37it/s]


In [ ]:
tf_idf_res[0]

In [19]:
for vuln, res in zip(vulns, tf_idf_res):
    vuln['top_k'] = [{'lib_name': lib, 'website_description':\
                      pros_mapping['token'][lib]} for lib in res]
    vuln['raw_label'] = vuln['label']
    vuln['labels'] = get_c_artifact(vuln['label'])
    del vuln['label']

In [148]:
for idx, vuln in enumerate(vulns):
    vuln['labels'] = [vuln['labels']]

In [149]:
output_dir = '/home/chentianyu/dependency/inputs/'
train_path = os.path.join(output_dir, 'train.json')
valid_path = os.path.join(output_dir, 'valid.json')
test_path = os.path.join(output_dir, 'test.json')

new_train = [vuln for vuln, label in zip(trains, vuln_labels[:-230]) if label in pros_names]
new_test = [vuln for vuln, label in zip(tests, vuln_labels[-230:]) if label in pros_names]

with open(train_path, 'w') as f:
    json.dump(new_train, f)

with open(valid_path, 'w') as f:
    json.dump(new_test, f)

with open(test_path, 'w') as f:
    json.dump(new_test, f)

In [143]:
len(new_test)

136

In [89]:
'ntp' in search_engine.search_topk_objects(cleaned_text(trains[0]['desc']), [])

True